In [1]:
# -*-conding: utf8 -*-
# !/usr/bin/python
# Author: Selvaria

# 稠密网络

import tensorflow as tf
import numpy as np
print(tf.__version__)

D:\Selvaria\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


2.0.0


In [2]:
# DenseNet使用了ResNet改良版的“批量归一化、激活和卷积”结构，我们首先在BottleNeck函数里实现这个结构。
# 在前向计算时，我们将每块的输入和输出在通道维上连结。

import tensorflow as tf

class BottleNeck(tf.keras.layers.Layer):
    def __init__(self, growth_rate, drop_rate):
        super(BottleNeck, self).__init__()
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filters=4 * growth_rate,
                                            kernel_size=(1, 1),
                                            strides=1,
                                            padding="same")
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters=growth_rate,
                                            kernel_size=(3, 3),
                                            strides=1,
                                            padding="same")
        self.dropout = tf.keras.layers.Dropout(rate=drop_rate)

        self.listLayers = [self.bn1,
                           tf.keras.layers.Activation("relu"),
                           self.conv1,
                           self.bn2,
                           tf.keras.layers.Activation("relu"),
                           self.conv2,
                           self.dropout]

    def call(self, x):
        y = x
        for layer in self.listLayers.layers:
            y = layer(y)
        y = tf.keras.layers.concatenate([x,y], axis=-1)
        return y

In [3]:
# 稠密块由多个BottleNeck组成，每块使用相同的输出通道数

class DenseBlock(tf.keras.layers.Layer):
    def __init__(self, num_layers, growth_rate, drop_rate=0.5):
        super(DenseBlock, self).__init__()
        self.num_layers = num_layers
        self.growth_rate = growth_rate
        self.drop_rate = drop_rate
        self.listLayers = []
        for _ in range(num_layers):
            self.listLayers.append(BottleNeck(growth_rate=self.growth_rate, drop_rate=self.drop_rate))

    def call(self, x):
        for layer in self.listLayers.layers:
            x = layer(x)
        return x

In [12]:
m = tf.keras.models.Sequential()
m.add(tf.keras.layers.Conv2D(filters=10, kernel_size=(3, 3), strides=1, padding="same", input_shape=(28,28,1)))
m.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 28, 28, 10)        100       
Total params: 100
Trainable params: 100
Non-trainable params: 0
_________________________________________________________________


In [4]:
blk = DenseBlock(2, 10)
X = tf.random.uniform((4, 8, 8,3))
Y = blk(X)
print(Y.shape)

(4, 8, 8, 23)


In [10]:
# 5.12.2 过渡层
# 过渡层用来控制模型复杂度。它通过1×1卷积层来减小通道数，并使用步幅为2的平均池化层减半高和宽，从而进一步降低模型复杂度。

class TransitionLayer(tf.keras.layers.Layer):
    def __init__(self, out_channels):
        super(TransitionLayer, self).__init__()
        self.bn = tf.keras.layers.BatchNormalization()
        self.conv = tf.keras.layers.Conv2D(filters=out_channels,
                                           kernel_size=(1, 1),
                                           strides=1,
                                           padding="same")
        self.pool = tf.keras.layers.MaxPool2D(pool_size=(2, 2),
                                              strides=2,
                                              padding="same")

    def call(self, inputs):
        x = self.bn(inputs)
        x = tf.keras.activations.relu(x)
        x = self.conv(x)
        x = self.pool(x)
        return x
    
blk = TransitionLayer(10)
blk(Y).shape

TensorShape([4, 4, 4, 10])

In [13]:
# 实现

num_init_features=64
growth_rate=32 
block_layers=[4,4,4,4]
# 此时第一次经过DenseNet的向前运算输出通道数为64+32*4 = 192，每次增加的通道数为32*4=128

compression_rate=0.5
drop_rate=0.5

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=num_init_features,kernel_size=(7, 7),strides=2, padding="same", input_shape=(28,28,1)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=2,  padding="same"))

num_channels = num_init_features
model.add(DenseBlock(num_layers=block_layers[0], growth_rate=growth_rate, drop_rate=drop_rate))
num_channels += growth_rate * block_layers[0]
num_channels = compression_rate * num_channels
model.add(TransitionLayer(out_channels=int(num_channels)))

model.add(DenseBlock(num_layers=block_layers[1], growth_rate=growth_rate, drop_rate=drop_rate))
num_channels += growth_rate * block_layers[1]
num_channels = compression_rate * num_channels
model.add(TransitionLayer(out_channels=int(num_channels)))

model.add(DenseBlock(num_layers=block_layers[2], growth_rate=growth_rate, drop_rate=drop_rate))
num_channels += growth_rate * block_layers[2]
num_channels = compression_rate * num_channels
model.add(TransitionLayer(out_channels=int(num_channels)))

model.add(DenseBlock(num_layers=block_layers[3], growth_rate=growth_rate, drop_rate=drop_rate))

model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(10, activation=tf.keras.activations.softmax))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 14, 14, 64)        3200      
_________________________________________________________________
batch_normalization_5 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
dense_block_1 (DenseBlock)   (None, 7, 7, 192)         209280    
_________________________________________________________________
transition_layer_1 (Transiti (None, 4, 4, 96)          19296     
_________________________________________________________________
dense_block_2 (DenseBlock)   (None, 4, 4, 224)         226176    
_________________________________________________________________
transition_layer_2 (Transiti (None, 2, 2, 112)        

In [14]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train.reshape((60000, 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=5,
                    validation_split=0.2)
test_scores = model.evaluate(x_test, y_test, verbose=2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 146s 3ms/sample - loss: 0.7162 - accuracy: 0.7302 - val_loss: 0.4562 - val_accuracy: 0.8381
Epoch 2/5
48000/48000 [==============================] - 139s 3ms/sample - loss: 0.4000 - accuracy: 0.8524 - val_loss: 0.4215 - val_accuracy: 0.8529
Epoch 3/5
48000/48000 [==============================] - 140s 3ms/sample - loss: 0.3365 - accuracy: 0.8764 - val_loss: 0.3387 - val_accuracy: 0.8773
Epoch 4/5
48000/48000 [==============================] - 142s 3ms/sample - loss: 0.3048 - accuracy: 0.8871 - val_loss: 0.3313 - val_accuracy: 0.8807
Epoch 5/5
48000/48000 [==============================] - 151s 3ms/sample - loss: 0.2828 - accuracy: 0.8950 - val_loss: 0.3079 - val_accuracy: 0.8909
10000/1 - 6s - loss: 0.3374 - accuracy: 0.8858
